In [ ]:
import numpy as np

In [ ]:
def gini_impurity(labels):
    # When the set is empty, it is also pure
    if labels.size == 0:
        return 0
    
    counts = np.unique(labels, return_counts = True)[1]
    fractions = counts/float(len(labels))
    
    return 1 - np.sum(fractions ** 2)

In [ ]:
def entropy(labels):
    # When the set is empty, it is also pure
    if labels.size == 0:
        return 0
    
    counts = np.unique(labels, return_counts = True)[1]
    fractions = counts/float(len(labels))
    
    return - np.sum(fractions * np.log2(fractions))

In [ ]:
# Do not modify this cell
criterion_function = {'gini': gini_impurity, 'entropy': entropy}
def weighted_impurity(groups, criterion='gini'):
    """ Calculate weighted impurity of children after a split
    Args:
        groups (list of children, and a child consists a list of class labels)
        criterion (metric to measure the quality of a split, 'gini' for Gini Impurity or 'entropy' for Information Gain)
    Returns:
        float, weighted impurity
    """
    total = sum(len(group) for group in groups)
    weighted_sum = 0.0
    for group in groups:
        weighted_sum += len(group) / float(total) * criterion_function[criterion](group)
    return weighted_sum

In [ ]:
def split_node(X, y, index, value):
    """ Split data set X, y based on a feature and a value
    Args:
        X, y (numpy.ndarray, data set)
        index (int, index of the feature used for splitting)
        value (value of the feature used for splitting)
    Returns:
        list, list: left and right child, a child is in the format of [X, y]
    """
    x = X[:, index]
    mask = x==value
    left = [X[~mask, :], y[~mask]]
    right = [X[mask], y[mask]]
    
    return left, right

In [ ]:
def get_best_split(X, y, criterion):
    """ Obtain the best splitting point and resulting children for the data set X, y
    Args:
        X, y (numpy.ndarray, data set)
        criterion (gini or entropy)
    Returns:
        dict {index: index of the feature, value: feature value, children: left and right children}
    """
    best_index, best_value, best_score, children = None, None, 1, None
    for index in range(len(X[0])):
        for value in np.sort(np.unique(X[:, index])):
          left_right = split_node(X, y, index, value)
          score = weighted_impurity([left_right[0][1], left_right[1][1]], criterion)
          if score < best_score : 
            best_index, best_value, best_score, children = index, value, score, left_right 

    return {'index': best_index, 'value': best_value, 'children': children}

In [ ]:
def get_leaf(labels):
    # Obtain the leaf as the majority of the labels
    return np.bincount(labels).argmax()

In [ ]:
def split(node, max_depth, min_size, depth, criterion):
    """ Split children of a node to construct new nodes or assign them terminals
    Args:
        node (dict, with children info)
        max_depth (int, maximal depth of the tree)
        min_size (int, minimal samples required to further split a child)
        depth (int, current depth of the node)
        criterion (gini or entropy)
    """
    left, right = node['children']
    del (node['children'])
    if left[1].size == 0:
        node['right'] = get_leaf(right[1])
        return
    if right[1].size == 0:
        node['left'] = get_leaf(left[1])
        return

    # Check if the current depth exceeds the maximal depth
    if depth >= max_depth:
        node['left'], node['right'] = get_leaf(left[1]), get_leaf(right[1])
        return

    # Check if the left child has enough samples
    if left[1].size <= min_size:
        node['left'] = get_leaf(left[1])
    else:
        next_left_right = get_best_split(left[0], left[1], criterion)
        next_left, next_right = next_left_right['children']
        if next_left[1].size == 0:
            node['left'] = get_leaf(next_right[1])
        elif next_right[1].size == 0:
            node['left'] = get_leaf(next_left[1])
        else:
            node['left'] = next_left_right
            split(node['left'], max_depth, min_size, depth + 1, criterion)

    # Check if the right child has enough samples
    if right[1].size <= min_size:
        node['right'] = get_leaf(right[1])
    else:
        next_left_right = get_best_split(right[0], right[1], criterion)
        next_left, next_right = next_left_right['children']
        if next_left[1].size == 0:
            node['right'] = get_leaf(next_right[1])
        elif next_right[1].size == 0:
            node['right'] = get_leaf(next_left[1])
        else:
            node['right'] = next_left_right
            split(node['right'], max_depth, min_size, depth + 1, criterion)

In [ ]:
def train_tree(X_train, y_train, max_depth, min_size, criterion='gini'):
    """ Construction of a tree starts here
    Args:
        X_train,  y_train (list, list, training data)
        max_depth (int, maximal depth of the tree)
        min_size (int, minimal samples required to further split a child)
        criterion (gini or entropy)
    """
    X = np.array(X_train)
    y = np.array(y_train)
    root = get_best_split(X, y, criterion)
    split(root, max_depth, min_size, 1, criterion)
    return root

In [ ]:
# Do not modify this cell
CONDITION = {'numerical': {'yes': '>=', 'no': '<'},
             'categorical': {'yes': 'is', 'no': 'is not'}}
def visualize_tree(node, depth=0):
    if isinstance(node, dict):
        if node['value'].dtype.kind in ['i', 'f']:
            condition = CONDITION['numerical']
        else:
            condition = CONDITION['categorical']
        print('{}|- X{} {} {}'.format(depth * '  ', node['index'] + 1, condition['no'], node['value']))
        if 'left' in node:
            visualize_tree(node['left'], depth + 1)
        print('{}|- X{} {} {}'.format(depth * '  ', node['index'] + 1, condition['yes'], node['value']))
        if 'right' in node:
            visualize_tree(node['right'], depth + 1)
    else:
        print('{}[{}]'.format(depth * '  ', node))

In [ ]:
# Do not modify this cell
X_train = [['tech', 'professional'],
           ['fashion', 'student'],
           ['fashion', 'professional'],
           ['sports', 'student'],
           ['tech', 'student'],
           ['tech', 'retired'],
           ['sports', 'professional']]

y_train = [1,
           0,
           0,
           0,
           1,
           0,
           1]

tree = train_tree(X_train, y_train, 2, 2)

# Expected Output
# |- X1 is not fashion
#   |- X2 is not professional
#     [0]
#   |- X2 is professional
#     [1]
# |- X1 is fashion
#   [0]
visualize_tree(tree)

|- X1 is not fashion
  |- X2 is not professional
    [0]
  |- X2 is professional
    [1]
|- X1 is fashion
  [0]
